<img src="../../img/elevation.png" alt="" style="width: 100%; border-radius: 20px;"/>

In [1]:
%%HTML
<style>
    body {
        --vscode-font-family: "Itim";
        font color: #d8c0b5;
    }
</style>

## Objective
This notebook aims to compute the elevation of each bird sighting in the bird sightings provided from *ornitho.ch* and *ornitho.de*, based on the latitude and longitude coordinates.

We expect the incorporation of altitude information to provide additional decision support for our deployed models that detect implausible bird sightings. For instance, certain bird species are typically associated with specific altitude ranges; therefore, altitude data can serve as a valuable feature for validating the authenticity of a bird sighting.

## Scope
In this notebook, we aim to generate altitude values for bird sightings in Germany and Switzerland. The altitude values are inherently only given for the Swiss datasets. Based on the latitude and longitude values of the bird sighting, we can extract them for Germany as well. 

For this, three distinct approaches for computing altitude values were investigated:
- **API-based**
    - *api.open-elevation.com* and *api.opentopodata.org*: Utilizing an external API to obtain altitude information.
    - Earth Engine: Leveraging Earth Engine data for altitude calculations.
- **Local**
    - DEM file: Employing a local Digital Elevation Model (EU_DEM tif file) for altitude computation.

After evaluating the efficiency and accuracy of the three approaches, the local DEM file method was selected as the preferred choice due to its computational speed and satisfactory performance in altitude calculation.

Additionally, an initial assessment is provided to determine the possible effectiveness of altitude values in supporting the detection of implausible bird data. In this context, we investigate the distinctive altitude ranges of two exemplatory bird species, i.e. the water pipit and White-tailed eagle.

## Output data
The resulting master dataset with elevation data is stored as selected_bird_species_with_grids.csv [here](https://drive.google.com/drive/folders/18XoTsDtWnN4QdIBNGGbq-jaa6U3nVb2e).

In [1]:
import sys
sys.path.append('../')

import pandas as pd
import ee
from utils.copernicus import CopernicusDEM
import requests

import plotly.express as px
import plotly.graph_objects as go
import geopandas as gpd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', False)

# 
<p style="background-color:#4A3228;color:white;font-size:240%;border-radius:10px 10px 10px 10px;"> &nbsp; 0️⃣ Specify your paths </p>

In order to run the notebook, the following file is required:
- Master dataset reduced to 27 species and enriched with EEA grids: *[selected_species_with_50km_grids.csv](https://drive.google.com/drive/folders/18XoTsDtWnN4QdIBNGGbq-jaa6U3nVb2e)*
- European Digital Elevation Model (EU-DEM); downloadable [here](https://www.eea.europa.eu/en/datahub/datahubitem-view/d08852bc-7b5f-4835-a776-08362e2fbf4b). It should contain the files *eu_dem_v11_E40N20.TIF* and *eu_dem_v11_E40N30.TIF*

If you wish to store the resulting dataframe enriched with elevation data, please specify a target path where it should be stored.

In [3]:
df_path = r'D:\\Simon\\Documents\\GP\\data\\datasets\\selected_bird_species_with_grids_50km.csv'  # please provide path to selected_bird_species_with_grids_50km.csv
eu_dem_path = r'D:\\Simon\\Documents\\GP\\data\\util_files\\EU_DEM\\'  # please provide path to the EU_DEM folder
target_path = r'D:\\Simon\\Documents\\GP\\data\\datasets\\selected_bird_species_50km_elevation.csv'  # please provide path where to save the dataset enriched with elevation data

# 
<p style="background-color:#4A3228;color:white;font-size:240%;border-radius:10px 10px 10px 10px;"> &nbsp; 1️⃣ Load data </p>

In this notebook, we compute the elevation data for all given sightings from 2018-2022.

There is already a column `altitude` which provide elevation information for all sightings in Switzerland. The German dataset only contains NaN values.

Although altitude values are present in the Swiss dataset, they are recalculated to ensure consistency. Therefore, the `altitude` column is dropped.

In [4]:
df = pd.read_csv(df_path, index_col=0, low_memory=False)
df.drop(columns=['altitude'], inplace=True)
df.head(5)

,id_sighting,id_species,name_species,date,timing,coord_lat,coord_lon,precision,total_count,atlas_code,id_observer,country,eea_grid_id
0,29666972,8.0,Haubentaucher,2018-01-01,NaN,53.157760,8.676993,place,0.0,0,37718.0,de,50kmE4200N3300
1,29654244,397.0,Schwarzkehlchen,2018-01-01,NaN,53.127639,8.957263,square,2.0,0,37803.0,de,50kmE4250N3300
2,29654521,463.0,Wiesenpieper,2018-01-01,NaN,50.850941,12.146953,place,2.0,0,39627.0,de,50kmE4450N3050
3,29666414,8.0,Haubentaucher,2018-01-01,NaN,51.076006,11.038316,place,8.0,0,38301.0,de,50kmE4350N3100
4,29656211,8.0,Haubentaucher,2018-01-01,NaN,51.389380,7.067282,place,10.0,0,108167.0,de,50kmE4100N3100


# 
<p style="background-color:#4A3228;color:white;font-size:240%;border-radius:10px 10px 10px 10px;"> &nbsp; 2️⃣ Elevation generation </p>

To calculate the altitude for each bird sighting based on the bird's location, three distinct approaches are employed:



The first two approaches involve API-based methods, specifically utilizing public API's specializing on altitude data, as well as Google Earth Engine, leveraging external online resources for altitude information. 

The third approach involves local computation, utilizing a local Digital Elevation Model (DEM) file, specifically the *EU_DEM* from the European Environment Agency*, for altitude calculations.

## 1st approach: Public API's with no registration requirement
*api.open-elevation.com* and *api.opentopodata.org* are prominent external APIs employed for obtaining elevation data in scientific and geospatial analyses. They do not require any authentication or registration, making it accessible for other developers to run this script.

Api.open-elevation.com serves as an elevation data service, offering a straightforward API for querying elevation information based on latitude and longitude coordinates. It utilizes global datasets to provide elevation values, making it accessible for a wide range of locations. 

On the other hand, api.opentopodata.org is a versatile API designed to retrieve various topographic data, including elevation, slope, and aspect. It supports multiple data sources and formats, allowing users to customize queries based on specific topographic parameters.

### Generation
In order to generate elevation data, both API's require the latitude and longitude value of the bird sighting. As there is so option to pass a batch or list of multiple sightings at once, the API's must be queried once for each sighting.

In [32]:
def get_elevation_openelevation(lat, long):
    query = f'https://api.open-elevation.com/api/v1/lookup?locations={lat},{long}'
    r = requests.get(query).json()
    elevation = r['results'][0]['elevation']
    return elevation

In [22]:
def get_elevation_opentopo(lat, long):
    query = f'https://api.opentopodata.org/v1/eudem25m?locations={lat},{long}'
    r = requests.get(query).json()
    elevation = r['results'][0]['elevation']
    return elevation

In [9]:
df['altitude'] = df.head(10).apply(lambda row: get_elevation_opentopo(row['coord_lat'], row['coord_lon']), axis=1)
df.head(5)

,id_sighting,id_species,name_species,date,timing,coord_lat,coord_lon,precision,total_count,atlas_code,id_observer,country,eea_grid_id,altitude
0,29666972,8.0,Haubentaucher,2018-01-01,NaN,53.157760,8.676993,place,0.0,0,37718.0,de,50kmE4200N3300,-1.051010
1,29654244,397.0,Schwarzkehlchen,2018-01-01,NaN,53.127639,8.957263,square,2.0,0,37803.0,de,50kmE4250N3300,0.782160
2,29654521,463.0,Wiesenpieper,2018-01-01,NaN,50.850941,12.146953,place,2.0,0,39627.0,de,50kmE4450N3050,270.566223
3,29666414,8.0,Haubentaucher,2018-01-01,NaN,51.076006,11.038316,place,8.0,0,38301.0,de,50kmE4350N3100,158.941010
4,29656211,8.0,Haubentaucher,2018-01-01,NaN,51.389380,7.067282,place,10.0,0,108167.0,de,50kmE4100N3100,52.463722


In [10]:
df['altitude'] = df.head(10).apply(lambda row: get_elevation_openelevation(row['coord_lat'], row['coord_lon']), axis=1)
df.head(5)

,id_sighting,id_species,name_species,date,timing,coord_lat,coord_lon,precision,total_count,atlas_code,id_observer,country,eea_grid_id,altitude
0,29666972,8.0,Haubentaucher,2018-01-01,NaN,53.157760,8.676993,place,0.0,0,37718.0,de,50kmE4200N3300,-1.0
1,29654244,397.0,Schwarzkehlchen,2018-01-01,NaN,53.127639,8.957263,square,2.0,0,37803.0,de,50kmE4250N3300,0.0
2,29654521,463.0,Wiesenpieper,2018-01-01,NaN,50.850941,12.146953,place,2.0,0,39627.0,de,50kmE4450N3050,268.0
3,29666414,8.0,Haubentaucher,2018-01-01,NaN,51.076006,11.038316,place,8.0,0,38301.0,de,50kmE4350N3100,157.0
4,29656211,8.0,Haubentaucher,2018-01-01,NaN,51.389380,7.067282,place,10.0,0,108167.0,de,50kmE4100N3100,51.0


### Issues
The public APIs, *api.open-elevation.com* and *api.opentopodata.org*, deliver accurate results for the dataset. Yet, with a growing number of API calls, the time taken for each call also increases gradually. As a result, for the extensive dataset at hand, using either of these APIs becomes impractical due to the extended processing time for each data point.

Therefore, the public API's are considered not feasible for generation elevation data for the bird sightings. 

## 2nd approach: *Earth Engine*
<img src="https://www.appgeo.com/wp-content/uploads/GoogleEarthEngine.png" alt="Google Earth Engine Icon" style="border-radius: 7px; width: 400px; height: auto;">

Earth Engine is a cloud-based platform developed by Google that provides access to a vast amount of geospatial data and computational resources for analyzing and visualizing Earth's surface.

In the context of extracting elevation data based on latitude and longitude values for Europe, Earth Engine can be utilized to access and process high-resolution digital elevation models (DEMs) and terrain data. DEMs represent the topography of the Earth's surface, providing information about elevation at different geographic locations.

Contrary to *api.open-elevation.com* and *api.opentopodata.org*, Earth Engine utilizes Google's cloud infrastructure, providing significant computational resources for large-scale geospatial analyses. It is designed to handle massive datasets and complex computations efficiently, which may leverage the scalability issues that emerged when using *api.open-elevation.com* and *api.opentopodata.org*.


### Generation
The process of querying an altitude value for a sighting is similar to the the public API's *api.open-elevation.com* and *api.opentopodata.org* - Earth Engine also requires separate API requests for each individual bird sighting.

<span style="color:red;">In order to query the Earth Engine, you must... [describe registration and authentication]</span>

In [12]:
ee.Authenticate()
ee.Initialize(project='ee-simonbirk')

def get_elevation_ee(lat, lon):
    point = ee.Geometry.Point([lon, lat])
    dataset = ee.Image('USGS/SRTMGL1_003')
    elevation = dataset.sample(point, 30).first().get('elevation').getInfo()
    return elevation

In [13]:
df.loc[:, 'altitude'] = df.head(10).apply(lambda row: get_elevation_ee(row['coord_lat'], row['coord_lon']), axis=1)
df.head(5)

,id_sighting,id_species,name_species,date,timing,coord_lat,coord_lon,precision,total_count,atlas_code,id_observer,country,eea_grid_id,altitude
0,29666972,8.0,Haubentaucher,2018-01-01,NaN,53.157760,8.676993,place,0.0,0,37718.0,de,50kmE4200N3300,-1.0
1,29654244,397.0,Schwarzkehlchen,2018-01-01,NaN,53.127639,8.957263,square,2.0,0,37803.0,de,50kmE4250N3300,0.0
2,29654521,463.0,Wiesenpieper,2018-01-01,NaN,50.850941,12.146953,place,2.0,0,39627.0,de,50kmE4450N3050,269.0
3,29666414,8.0,Haubentaucher,2018-01-01,NaN,51.076006,11.038316,place,8.0,0,38301.0,de,50kmE4350N3100,157.0
4,29656211,8.0,Haubentaucher,2018-01-01,NaN,51.389380,7.067282,place,10.0,0,108167.0,de,50kmE4100N3100,51.0


### Issues
The shift from the two public API's to Earth Engine was expected to address previously encountered computational inefficiencies. Despite Earth Engine providing a robust geospatial analysis platform with extensive computational resources, the processing duration remains too high for our use case.

## 3rd approach: *European Digital Elevation Model* by EEA

<img src="https://www.eea.europa.eu/logo.jpg" alt="EEA Icon" style="border-radius: 7px; width: 400px; height: auto;">


Due to prolonged computation times experienced with both registration-free API's and Earth Engine in extracting elevation data, it is opted to transition to a local computation approach. Despite the necessity to download files for this method, it holds the potential to address the issue of excessive processing times, as computations are now conducted locally rather than through an API.

For this purpose, we are utilizing *EU_DEM* files. The EU_DEM (European Digital Elevation Model) is provided by the European Environment Agency (EEA). The EEA is an agency of the European Union that focuses on providing environmental information to support policy development and implementation in Europe. It is part of the Copernicus program, specifically managed by the Copernicus Land Monitoring Service (CLMS) [1]. 

The elevation data in the EU_DEM is derived from various sources, including satellite data and airborne measurements [1], and is processed to create a high-quality digital representation of the Earth's surface for the European continent. It is freely available for download and use.



### Generation
To retrieve the elevation data, EEA provides a module that allows to compute all elevation data for a dataframe at once. For more detailed code information, please visit *../utils/copernicus.py*.

In [14]:
copernicus = CopernicusDEM(raster_paths=[eu_dem_path + 'eu_dem_v11_E40N20.TIF', eu_dem_path + 'eu_dem_v11_E40N30.TIF'])
df = copernicus.get_elevation(df, lat_col='coord_lat', lon_col='coord_lon')

In [15]:
df['altitude'] = df['elevation']
df.drop(columns=['elevation'], inplace=True)

### Issues

#### Issue: Incorrect altitude values due to EU_DEM borders

In [16]:
print(df.altitude.value_counts())

altitude
-3.402823e+38    51606
 3.913700e+02    33840
 4.265330e+02    28527
 3.182081e+02    12550
 3.718830e+02    9445 
                 ...  
 8.830636e+02    1    
 9.352755e+02    1    
 7.191224e+02    1    
 9.459383e+02    1    
 8.021597e+02    1    
Name: count, Length: 910675, dtype: int64


In [17]:
min_altitude = df.altitude.min()
df_min_altitude = df[df.altitude == min_altitude]
df_min_altitude.head(5)

,id_sighting,id_species,name_species,date,timing,coord_lat,coord_lon,precision,total_count,atlas_code,id_observer,country,eea_grid_id,altitude
56,29660454,8.0,Haubentaucher,2018-01-01,NaN,53.780461,7.947729,precise,1.0,0,77774.0,de,50kmE4150N3400,-3.402823e+38
131,29652796,158.0,Seeadler,2018-01-01,NaN,53.702563,7.740776,square,1.0,0,43003.0,de,50kmE4150N3400,-3.402823e+38
278,29667163,8.0,Haubentaucher,2018-01-01,NaN,54.505650,9.553208,precise,2.0,0,37768.0,de,50kmE4250N3450,-3.402823e+38
301,29655667,8.0,Haubentaucher,2018-01-01,02:44,54.758682,8.295368,precise,1.0,0,42717.0,de,50kmE4200N3500,-3.402823e+38
302,29653528,8.0,Haubentaucher,2018-01-01,03:14,54.758271,8.294853,precise,2.0,0,45902.0,de,50kmE4200N3500,-3.402823e+38


In [18]:
sample_rows = df_min_altitude.sample(n=100, random_state=1)

In [19]:
layout = go.Layout(mapbox=dict(style='open-street-map', zoom=10), showlegend=False, margin=dict(l=0, r=0, t=0, b=0))
fig = go.Figure(layout=layout)

def add_trace(row):
    bird_sighting = row.copy()
    bird_sighting['color'] = '#d00000'
    geometry = gpd.points_from_xy([bird_sighting['coord_lon']], [bird_sighting['coord_lat']])
    gdf_data_point = gpd.GeoDataFrame([bird_sighting], geometry=geometry, crs='EPSG:4326')

    fig.add_trace(px.scatter_mapbox(gdf_data_point,
                    lat='coord_lat', lon='coord_lon', size_max=10, color_discrete_sequence=[gdf_data_point.color],
                    hover_name=gdf_data_point['name_species']).data[0]) 

sample_rows.apply(add_trace, axis=1)

fig.update_layout(mapbox_style='open-street-map', mapbox_zoom=5, mapbox_center={'lat': 55, 'lon': 10})
fig.show()


#### Solution: Incorporate one of the above mentioned APIs
We can generate the elevation data agin for those values which were calculated not correctly.
Later on we can replace them in the dataframe

In [33]:
# runtime for 50k columns: about 3h
df_min_altitude.loc[:, 'altitude'] = df_min_altitude.head(1000).apply(lambda row: get_elevation_ee(row['coord_lat'], row['coord_lon']), axis=1)

In [ ]:
df_min_altitude.tail(5)

,id_sighting,id_species,name_species,date,coord_lat,coord_lon,total_count,atlas_code,country,eea_grid_id,urban_area_percent,industrial_area_percent,agriculture_area_percent,forest_area_percent,grassland_area_percent,shrubland_area_percent,coastal_area_percent,rocky_area_percent,sparsley_vegetated_area_percent,burnt_area_percent,glacier_area_percent,wetlands_area_percent,water_area_percent,land_use_coord,land_use_coord_numeric,unclassified_area_percent,altitude
2018382,63621895,463,Wiesenpieper,2021-12-31,53.720317,7.407022,3,NaN,de,50kmE4100N3400,0.000000,0.0,0.000000,0.000000,0.000106,0.0,0.0,0.0,0.0,0.0,0.0,0.999894,0.000000,wetlands,13,0.0,None
2018528,74075515,52,Singschwan,2022-12-31,54.567683,13.407219,0,NaN,de,50kmE4500N3500,0.000000,0.0,0.000000,0.030988,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.969012,water,14,0.0,None
2018690,74096151,503,Berghänfling,2022-12-31,54.395487,8.658754,100,NaN,de,50kmE4200N3450,0.000000,0.0,0.336060,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.663940,0.000000,wetlands,13,0.0,None
2018692,74096147,158,Seeadler,2022-12-31,54.400809,8.658721,1,NaN,de,50kmE4200N3450,0.000000,0.0,0.006298,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.874969,0.118733,wetlands,13,0.0,None
2018722,74043701,52,Singschwan,2022-12-31,54.144432,13.630751,16,NaN,de,50kmE4550N3450,0.030445,0.0,0.000000,0.277297,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.692258,water,14,0.0,None


In [ ]:
df_min_altitude.isna().sum()

id_sighting                        0    
id_species                         0    
name_species                       0    
date                               0    
coord_lat                          0    
coord_lon                          0    
total_count                        0    
atlas_code                         48671
country                            0    
eea_grid_id                        0    
urban_area_percent                 0    
industrial_area_percent            0    
agriculture_area_percent           0    
forest_area_percent                0    
grassland_area_percent             0    
shrubland_area_percent             0    
coastal_area_percent               0    
rocky_area_percent                 0    
sparsley_vegetated_area_percent    0    
burnt_area_percent                 0    
glacier_area_percent               0    
wetlands_area_percent              0    
water_area_percent                 0    
land_use_coord                     0    
land_use_coord_n

In [ ]:
df.loc[df['altitude'] == df.altitude.min(), 'altitude'] = 0

In [ ]:
print(df.altitude.value_counts())

## Conclusion


## Outlook

# 
<p style="background-color:#4A3228;color:white;font-size:240%;border-radius:10px 10px 10px 10px;"> &nbsp; 3️⃣ Add elevation feature to the dataset</p>

In [16]:
df_path = 'D:\\Simon\\Documents\\GP\\data\\datasets\\selected_bird_species_with_grids_50km.csv'
df = pd.read_csv(df_path, index_col=0)
df.reset_index(inplace=True, drop=True)

C:\Users\simon\AppData\Local\Temp\ipykernel_17476\247923559.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_path, index_col=0)


In [17]:
df_path_alt = 'D:\\Simon\\Documents\\GP\\data\\datasets\\altitude\\dbird_export_altitude_202312140925.csv'
df_alt = pd.read_csv(df_path_alt, sep=';')

In [ ]:
df_alt.head(3)

In [18]:
df_de = df[df['country'] == 'de']

merged_df_de = df_de.merge(df_alt, left_on='id_sighting', right_on='ornitho_id', how='left')
merged_df_de.drop(columns=['ornitho_id', 'altitude_x'], inplace=True)
merged_df_de.rename(columns={'altitude_y': 'altitude'}, inplace=True)

df.update(merged_df_de)

In [19]:
df.head(3)

,id_sighting,id_species,name_species,date,timing,coord_lat,coord_lon,precision,altitude,total_count,atlas_code,id_observer,country,eea_grid_id
0,29666972,8.0,Haubentaucher,2018-01-01,NaN,53.157760,8.676993,place,-1,0.0,NaN,37718.0,de,50kmE4200N3300
1,29654244,397.0,Schwarzkehlchen,2018-01-01,NaN,53.127639,8.957263,square,1,2.0,NaN,37803.0,de,50kmE4250N3300
2,29654521,463.0,Wiesenpieper,2018-01-01,NaN,50.850941,12.146953,place,269,2.0,NaN,39627.0,de,50kmE4450N3050


In [20]:
print(df.altitude.value_counts())

altitude
 0       115450
 430     88178 
-1       51270 
-2       43926 
 400     38170 
         ...   
 2893    1     
 3262    1     
 3124    1     
 3001    1     
 2971    1     
Name: count, Length: 2899, dtype: int64


In [21]:
min_altitude = df.altitude.min()
df_min_altitude = df[df.altitude == min_altitude]
df_min_altitude.head(5)

,id_sighting,id_species,name_species,date,timing,coord_lat,coord_lon,precision,altitude,total_count,atlas_code,id_observer,country,eea_grid_id
172472,57776355,463.0,Wiesenpieper,2021-05-03,12:08,50.958478,6.674523,precise,-34,1.0,NaN,84909.0,de,50kmE4050N3050
172473,57790281,463.0,Wiesenpieper,2021-05-03,12:08,50.958478,6.674523,precise,-34,1.0,NaN,88575.0,de,50kmE4050N3050


In [23]:
df_path = 'D:\\Simon\\Documents\\GP\\data\\datasets\\selected_bird_species_with_grids_50km.csv'
# df_path = 'D:\\Simon\\Documents\\GP\\data\\datasets\\selected_bird_species_50km_elevation.csv'
df.to_csv(df_path)

# 
<p style="background-color:#4A3228;color:white;font-size:240%;border-radius:10px 10px 10px 10px;"> &nbsp; 4️⃣ Initial assessment of the elevation feature: Possible degree of decision support </p>

Water pipit und Seeadler -> x-axis datum, y-axis elevatino; daran kann man sehen, dass species-specific elevation patterns auftreten; z.B. water pipit: fliegt im Sommer in höhere Lagen; Seeadler ist immer niedrig; daher kann man ausreißer schon an der elevation data allein feststellen, d.h. das könnte ein wichtiges feature in der outlier detection sein.

# References
[1]  European Environment Agency. "European Digital Elevation Model (EU-DEM), Version 1.1." (2016).